<b>
    
<p>
<center>
<font size="5">
Content-Based Filtering
</font>
</center>
    <center>
<font size="4">
Content-Based Filtering using TF-IDF and Cosine Similarity
</font>
</center>
</p>
   
</b>

<p><h2>Table of Contents<span class="tocSkip"></span></h2></p>

<div class="toc"><ul class="toc-item">
    <li><span><a ><span>1&nbsp;&nbsp;</span>Overview</a></span></li>
    <li><span><a ><span>2&nbsp;&nbsp;</span>Import Library</a></span></li>
    <li><span><a ><span>3&nbsp;&nbsp;</span>Load Data</a></span></li>
    <li><span><a ><span>4&nbsp;&nbsp;</span>Tạo Vector đặc trưng cho mỗi bộ phim</a></span>
        <ul class="toc-item">
            <li><span><a ><span>4.1&nbsp;&nbsp;</span>Tạo document cho mỗi bộ phim</a></span></li>
            <li><span><a ><span>4.2&nbsp;&nbsp;</span>Tính bagofword</a></span></li>
            <li><span><a ><span>4.3&nbsp;&nbsp;</span>Áp dụng TF-IDF cho document tạo vector cho từng bộ phim</a></span></li>
            </ul>
    </li>
    <li><span><a ><span>5&nbsp;&nbsp;</span>Xây dựng đề xuất phim dựa trên Cosine Siminarity</a></span>
        <ul class="toc-item">
            <li><span><a >5.1&nbsp;&nbsp;</span>Tính Cosine Siminarity</a></span></li>
            <li><span><a >5.2&nbsp;&nbsp;</span>Recommend top 20 phim có Cosine Similarity lớn nhất</a></span></li>
            <li><span><a ><span>5.3&nbsp;&nbsp;</span>Evaluation</a></span></li>
            </ul>
    </li>

### Overview

#### Aim
1. Tạo vector đặc trưng cho từng bộ phim dựa trên tag bằng kỹ thuật TF-IDF
2. Xây dựng recommend cho từng bộ phim dựa trên độ tương tự Cosine Siminarity 

### Import Library

In [1]:
import os
os.environ["SPARK_HOME"] = "/opt/spark"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre"

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.mllib.feature import Normalizer
from pyspark.mllib.feature import HashingTF, IDF

In [3]:


spark = SparkSession.builder \
    .appName("Project") \
    .master("local[*]") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()
sc = spark.sparkContext

23/05/27 13:09:43 WARN Utils: Your hostname, longcule-Inspiron-7591 resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface enx68e43b306c5a)
23/05/27 13:09:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/27 13:09:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load Data

In [4]:

mov = spark.read.csv('hdfs://127.0.0.1:9900/user/movies.csv', header=True,inferSchema="true")
mov.show()
display(mov)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

DataFrame[movieId: int, title: string, genres: string]

In [5]:

tags = spark.read.csv('hdfs://127.0.0.1:9900/user/tags_new.csv',header=True).drop('userId')
display(tags)

DataFrame[movieId: string, tag: string]

In [6]:

read_movies_1m = sc.textFile("hdfs://127.0.0.1:9900/user/movies.csv")
read_movies_1m_header = read_movies_1m.take(1)[0]
read_movies_1m_header

'movieId,title,genres'

In [7]:

s_df = spark.read.format("csv").option("header", "true").load('hdfs://127.0.0.1:9900/user/movies.csv')
a = [i.movieId for i in s_df.select('movieId').distinct().collect()]
print('Number of unique Movies')
len(a)

Number of unique Movies


58098

In [8]:

read_tags_1m = sc.textFile("hdfs://127.0.0.1:9900/user/tags_new.csv")
read_tags_1m_header = read_tags_1m.take(1)[0]
read_tags_1m_header

23/05/27 13:09:56 WARN BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	at org.apac

'movieId,tag'

### Tạo vector đặc trưng cho từng bộ phim

#### Tạo document cho mỗi bộ phim

In [9]:
#creation of document for each movie in movies tags
read_tags_1m_data = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:(int(tokens[0]),tokens[1]))\
.groupByKey().sortByKey()

documents = read_tags_1m_data.map(lambda x :   [i  for i in x[1]])

read_tags = spark.createDataFrame(read_tags_1m_data,('movieid_num', 'document'))
read_tags.show(10)

a = [i.movieid_num for i in read_tags.select('movieid_num').distinct().collect()]
display(read_tags)

+-----------+--------------------+
|movieid_num|            document|
+-----------+--------------------+
|          1|{[toy, adventure,...|
|          2|{[Kirsten Dunst, ...|
|          3|{[old, comedinha ...|
|          4|{[chick flick, ch...|
|          5|{[sequel fever, c...|
|          6|{[suspense, bank ...|
|          7|{[remake, remake,...|
|          8|{[Peter Hewitt, M...|
|          9|{[CLV, Peter Hyam...|
|         10|{[spies, violence...|
+-----------+--------------------+
only showing top 10 rows



DataFrame[movieid_num: bigint, document: struct<data:array<string>,index:bigint,maxindex:bigint>]

In [10]:
a.sort()
a.index(109487)
d1 = documents.collect()
d1[826]
a.index(236)

233

#### Tính bag of words

In [11]:

bagfwords = read_tags_1m.filter(lambda line : line != read_tags_1m_header).map(lambda line:line.split(",")).map(lambda tokens:((tokens[1]), int(1)))\
.reduceByKey(lambda x,y:x+y)
dfbg = spark.createDataFrame(bagfwords,('words','frequency'))
dfbg.show()
display(dfbg)
features_length = bagfwords.count()
print(bagfwords.count())

+--------------------+---------+
|               words|frequency|
+--------------------+---------+
|               Pixar|      965|
|            Engaging|        2|
|           animation|     2828|
|               pixar|      471|
|         buddy movie|      177|
|           Tim Allen|       60|
|              family|     1928|
|              sci-fi|     9402|
|    fun family movie|        5|
|              comedy|     5932|
|         time travel|     3491|
|          kids movie|       19|
|               dolls|       42|
|            jealousy|      552|
|             mission|       85|
|                Want|       69|
|     Pixar animation|       51|
|            BD-Video|      732|
|        imdb top 250|     1798|
|Tumey's To See Again|       21|
+--------------------+---------+
only showing top 20 rows



DataFrame[words: string, frequency: bigint]

105871


In [12]:
#tags count
read_tags_1m_data.count()

57716

#### TF-IDF Documents tạo vector đặc trưng

In [13]:

hashingTF = HashingTF(features_length)
tf = hashingTF.transform(documents)
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)



### Xây dựng hệ đề xuất phim

Tìm document của bộ phim được chỉ định

In [14]:
def get_movie_docbyid(movieid):
  return read_tags_1m_data.filter(lambda x: x[0] == movieid).map(lambda x :  [i  for i in x[1]])

In [15]:
new_movie = get_movie_docbyid(1)
print(new_movie.take(1))

[['toy', 'adventure', 'friendship', 'Pixar', 'Engaging', 'adventure', 'children', 'Pixar', 'kids', 'animation', 'computer animation', 'animation', 'Disney', 'friendship', 'pixar', 'buddy movie', 'computer animation', 'friendship', 'Tom Hanks', 'Tom Hanks', 'fun', 'buddy movie', 'Pixar', 'Pixar', 'animation', 'animation', 'Pixar', 'Pixar', 'animation', 'fantasy', 'Tim Allen', 'Tom Hanks', 'animation', 'family', 'sci-fi', 'animation', 'animated', 'fun family movie', 'comedy', 'children', 'Pixar', 'funny', 'time travel', 'witty', "erlend's DVDs", 'animation', 'animated', 'cgi', 'Disney', 'fantasy', 'friendship', 'cowboy', 'animation', 'animation', 'classic', 'computer animation', 'comedy', 'Disney', 'Pixar', 'funny', 'Disney', 'animation', 'animation', 'dinosaur', 'kids movie', 'animation', 'Pixar', 'dolls', 'adventure', 'feel-good', 'neighborhood', 'Disney', 'Pixar', 'classic', 'animation', 'ya boy', 'Pixar', 'Tim Allen', 'Tom Hanks', 'Disney', 'martial arts', 'pixar', 'jealousy', 'Pixar

Tính vector đặc trưng của bộ phim

In [16]:

candidateTf = hashingTF.transform(new_movie)
candidateTfIdf = idf.transform(candidateTf)

Chuyển đổi từ sprase vector sang dense vector

In [17]:

frequencyDenseVectors_0 = tfidf.map(lambda vector: DenseVector(vector.toArray()))
frequencyDenseVectors_1 =  candidateTfIdf.map(lambda vector: DenseVector(vector.toArray()))

#### Tính Cosine Similarity

In [18]:
y1 =   frequencyDenseVectors_1.collect()

re = frequencyDenseVectors_0.map(lambda x : (x.dot(y1[0]))/(x.norm(2)*y1[0].norm(2)))


#### Recommend top 20 phim có Cosine lớn nhất

In [19]:
result1=re.collect()
dict1 = {}
list_index  = []
for i in result1:
  if (i > 0.1)&(i<0.999999) :
    dict1[a[result1.index(i)]] = i
sorted_d = sorted(dict1.items(), key=lambda x: x[1])
top20_r  = sorted_d[-20:]
for i in top20_r:
  list_index.append(i[0])
list_index

[95856,
 5444,
 95313,
 72356,
 95654,
 95311,
 120474,
 68954,
 45517,
 95858,
 157296,
 50872,
 5218,
 103141,
 8961,
 4886,
 6377,
 78499,
 2355,
 3114]

In [37]:
top20_rec = sc.parallelize(top20_r)
df_top20 = spark.createDataFrame([[str(top20_r1[0]), float(top20_r1[1])] for top20_r1 in top20_r ],('movienum','similarity_score'))
df_top20.show()

+--------+-------------------+
|movienum|   similarity_score|
+--------+-------------------+
|   95856|0.41673111133016316|
|    5444|0.41806934006999824|
|   95313| 0.4217953961135016|
|   72356|  0.429207318048189|
|   95654|0.43608017435016416|
|   95311| 0.4449878514267786|
|  120474|0.45930223496399514|
|   68954|0.49971758417679235|
|   45517| 0.5079383367087488|
|   95858| 0.5127637382773084|
|  157296| 0.5391505121830925|
|   50872| 0.5405995988350474|
|    5218| 0.5573818082873833|
|  103141| 0.5643087087214725|
|    8961| 0.5733338677902318|
|    4886| 0.5747731376239061|
|    6377| 0.6536042277438002|
|   78499| 0.6583849972452238|
|    2355| 0.6712633982436051|
|    3114|  0.755207496722055|
+--------+-------------------+



In [35]:
df_top20.drop('similarity_score')
df_top20.show()

+--------+-------------------+
|movienum|   similarity_score|
+--------+-------------------+
|   95856|0.41673111133016316|
|    5444|0.41806934006999824|
|   95313| 0.4217953961135016|
|   72356|  0.429207318048189|
|   95654|0.43608017435016416|
|   95311| 0.4449878514267786|
|  120474|0.45930223496399514|
|   68954|0.49971758417679235|
|   45517| 0.5079383367087488|
|   95858| 0.5127637382773084|
|  157296| 0.5391505121830925|
|   50872| 0.5405995988350474|
|    5218| 0.5573818082873833|
|  103141| 0.5643087087214725|
|    8961| 0.5733338677902318|
|    4886| 0.5747731376239061|
|    6377| 0.6536042277438002|
|   78499| 0.6583849972452238|
|    2355| 0.6712633982436051|
|    3114|  0.755207496722055|
+--------+-------------------+



In [48]:
def get_movie_ids(movie_id):
    
    new_movie = get_movie_docbyid(movie_id)
    candidateTf = hashingTF.transform(new_movie)
    candidateTfIdf = idf.transform(candidateTf)
    
    frequencyDenseVectors_0 = tfidf.map(lambda vector: DenseVector(vector.toArray()))
    frequencyDenseVectors_1 =  candidateTfIdf.map(lambda vector: DenseVector(vector.toArray()))

    y1 =   frequencyDenseVectors_1.collect()
    re = frequencyDenseVectors_0.map(lambda x : (x.dot(y1[0]))/(x.norm(2)*y1[0].norm(2)))

    result1=re.collect()


    dict1 = {}
    list_index  = []
    for i in result1:
      if (i > 0.1)&(i<0.999999) :
        dict1[a[result1.index(i)]] = i
    sorted_d = sorted(dict1.items(), key=lambda x: x[1])
    top5_r  = sorted_d[-20:]
    for i in top5_r:
      list_index.append(i[0])

    return list_index

In [52]:
movi = 3
kq = get_movie_ids(movi)


In [53]:
print(kq)

[92176, 148294, 1095, 4509, 135741, 26818, 909, 8758, 4186, 134747, 174445, 124515, 6522, 6166, 3076, 1837, 1083, 3450, 50153, 7093]


### Evaluate

In [44]:
from pyspark.sql.functions import col
def get_movie_genres(movieId, movies_df):
    # Lấy danh sách thể loại của bộ phim dựa trên movieId
    genres = (
        movies_df
        .filter(col('movieId') == movieId)
        .select('genres')
        .rdd.flatMap(lambda row: row[0].split('|'))
        .collect()
    )
    
    return genres

#### Tính Hit Rate

In [60]:
def movie_evaluation(movieid, predicted_movies):
    main_movie_genre = get_movie_genres(movieid, mov)
    print(main_movie_genre)
    movie_genres_list = []
    for each in predicted_movies:
        genre_list_per_movie = get_movie_genres(each, mov)
        movie_genres_list.append(genre_list_per_movie)
    recc_list = []
    for each in movie_genres_list:
        for all in each:
            recc_list.append(all)
    print(recc_list)
    hit = 0
    fault = 0
    total = 0
    for i in recc_list:
        if i in main_movie_genre:
            hit += 1
        else:
            fault += 1
        total += 1
    return hit/total

In [68]:
from pyspark.sql.functions import rand

def calculate_average_hit_rate(num_movies):
    # Lấy một số lượng phim ngẫu nhiên từ DataFrame movies
    random_movies = mov.orderBy(rand()).limit(num_movies)
    
    total_hit_rate = 0.0
    for row in random_movies.collect():
        movie_id = row['movieId']
        predicted_movies = get_movie_ids(movie_id)
        hit_rate = movie_evaluation(movie_id, predicted_movies)
        total_hit_rate += hit_rate
    
    # Tính trung bình tỷ lệ hit rate
    average_hit_rate = total_hit_rate / num_movies
    
    return average_hit_rate


# Số lượng phim muốn lấy ngẫu nhiên
num_movies = 10

# Gọi hàm calculate_average_hit_rate
average_hit_rate = calculate_average_hit_rate(num_movies)

# In kết quả trung bình tỷ lệ hit rate
print('Average Hit Rate:', average_hit_rate)


['Action', 'Animation', 'Sci-Fi']
['Action', 'Animation', 'Sci-Fi', 'Western', 'Drama', 'Animation', 'Fantasy', 'Sci-Fi', 'Animation', 'Fantasy', 'Horror', 'Sci-Fi', 'Animation', 'Animation', 'Drama', 'Fantasy', 'Animation', 'Sci-Fi', 'Animation', 'Comedy', 'Drama', 'Action', 'Adventure', 'Animation', 'Fantasy', 'Sci-Fi', 'Action', 'Animation', 'Horror', 'Action', 'Adventure', 'Animation', 'Fantasy', 'Animation', 'Sci-Fi', 'Animation', 'Animation', 'Crime', 'Mystery', 'Sci-Fi', 'Thriller', '(no genres listed)', 'Animation', 'Romance', 'Action', 'Animation', 'Crime', 'Sci-Fi', 'Thriller', 'Animation', '(no genres listed)', 'Animation', 'Comedy']


['Drama', 'Fantasy', 'War']
['(no genres listed)', 'Comedy', 'Drama', 'Drama', 'Drama', 'Drama', 'Horror', 'Children', 'Comedy', 'Drama', 'Drama', 'Drama', 'Comedy', 'Action', 'Drama', 'Romance', 'War', 'Children', 'Comedy', 'Crime', 'Documentary', 'Comedy', '(no genres listed)', 'Comedy', 'Crime', 'Drama', 'War', 'Comedy', 'Drama', 'Romance', '(no genres listed)']


['Action', 'Drama', 'Thriller']
['(no genres listed)', '(no genres listed)', 'Horror', 'Thriller', '(no genres listed)', 'Action', 'Crime', 'Drama', 'Thriller', 'Drama', 'Documentary', '(no genres listed)', '(no genres listed)', 'Western', 'Action', 'Adventure', 'Thriller', 'War', 'Children', 'Fantasy', 'Drama', 'Horror', 'Action', 'Documentary', 'Crime', 'Action', 'Comedy', 'Thriller', 'Action', 'Adventure', 'Fantasy', 'Sci-Fi', 'Action', 'Comedy', 'Drama', 'Romance']


['Crime', 'Drama', 'Thriller']
['Action', 'Comedy', 'Western', 'Action', 'Comedy', 'Thriller', 'Crime', 'Drama', 'Mystery', 'Thriller', 'Crime', 'Comedy', 'Drama', 'Documentary', 'Drama', 'Comedy', 'Drama', 'Mystery', 'Western', 'Western', 'Action', 'Comedy', 'Crime', 'Horror', 'Thriller', 'Thriller', 'Documentary', 'Crime', 'Drama', 'Western', 'Crime', 'Drama']


['Drama', 'Romance']
['Adventure', 'Crime', 'Drama', 'Comedy', 'Drama', 'Romance', 'Horror', 'Musical', 'Drama', 'Children', 'Drama', 'Comedy', 'Drama', 'Adventure', 'Children', 'Fantasy', 'Animation', 'Children', 'Animation', 'Children', 'Comedy', 'Drama', 'Drama', 'Action', 'Crime', 'Drama', 'Crime', 'Drama', 'Drama', 'Fantasy', 'Animation', 'Children', '(no genres listed)', 'Animation', 'Drama', 'Romance', 'Drama', 'Children', 'Crime', 'Drama']


['Drama', 'Thriller']
['Documentary', 'Drama', 'Thriller', 'Action', 'Crime', 'Drama', 'Thriller', 'Comedy', 'Crime', 'Comedy', 'Horror', 'Documentary', 'Action', 'Comedy', 'Crime', 'Romance', 'Horror', 'Adventure', 'Documentary', 'IMAX', 'Documentary', 'Crime', 'Thriller', 'Comedy', '(no genres listed)', 'Documentary', 'Action', 'Comedy', 'Crime', 'Drama', 'Drama', 'Adventure', 'Drama', 'Romance', 'Drama', 'Crime']


['Comedy']
['Horror', 'Thriller', 'Horror', 'Comedy', 'Documentary', 'Comedy', 'Drama', 'Horror', 'Adventure', 'Thriller', 'Comedy', 'Horror', 'Comedy', 'Comedy', 'Horror', 'Comedy', 'Comedy', 'Crime', 'Drama', 'Thriller', 'Drama', 'Romance', 'Comedy', 'Horror', 'Action', 'Mystery', 'Sci-Fi', 'Thriller', 'Comedy', 'Comedy']


['Adventure']
['Drama', 'Drama', 'Drama', 'Romance', 'Drama', 'Romance', 'Sci-Fi', 'Thriller', 'Documentary', 'Drama', 'Thriller', 'Drama', 'Romance', 'Thriller', 'Comedy', 'Drama', 'Romance', 'Drama', 'Thriller', 'Crime', 'Drama', 'Romance', 'Thriller', 'Drama', 'Horror', 'Drama', 'Drama', 'Horror', 'Sci-Fi', 'Adventure', 'Drama', 'Drama', 'Comedy', 'Crime']


['Drama', 'Horror']
['Adventure', 'Drama', 'Romance', 'Horror', 'Drama', 'Drama', 'Drama', 'Crime', 'Drama', 'Thriller', 'Drama', 'Romance', 'Drama', 'Mystery', 'Thriller', 'Drama', 'Comedy', 'Drama', 'Drama', 'Drama', 'Drama', 'Drama', 'Romance', 'Crime', 'Horror', 'Drama', 'Drama', 'Romance', 'Drama', 'Drama', 'Romance', 'Drama']


['Comedy', 'Sci-Fi']
['Horror', 'Sci-Fi', 'Horror', 'Sci-Fi', 'Comedy', 'Drama', 'Fantasy', 'Horror', 'Sci-Fi', 'Action', 'Sci-Fi', 'Thriller', 'Horror', 'Sci-Fi', 'Action', 'Horror', 'Sci-Fi', 'Thriller', 'Horror', 'Horror', 'Horror', 'Thriller', 'Action', 'Horror', 'Sci-Fi', 'Horror', 'Fantasy', 'Sci-Fi', 'Comedy', 'Drama', 'Horror', 'Sci-Fi', 'Horror', 'Sci-Fi', 'Horror', 'Action', 'Adventure', 'Horror', 'Sci-Fi', 'Thriller', 'Horror', 'Sci-Fi', 'Fantasy', 'Horror']
Average Hit Rate: 0.3741700315152528


In [61]:
movieid = 3
hit_rate_movie = movie_evaluation(movieid, get_movie_ids(movieid))
print('The Hit Rate for this method is:', hit_rate_movie)

['Comedy', 'Romance']
['Comedy', 'Drama', 'Romance', 'Comedy', 'Romance', 'Drama', 'Comedy', 'Action', 'Comedy', 'Drama', 'Action', 'Adventure', 'Comedy', 'Comedy', 'Drama', 'Romance', 'Comedy', 'Drama', 'Comedy', 'Drama', 'Romance', 'Crime', 'Drama', 'Mystery', 'Romance', 'Thriller', 'Comedy', 'Horror', 'Comedy', 'Comedy', 'Comedy', 'Comedy', 'Comedy', 'Comedy', 'Musical', 'Comedy', 'Action', 'Comedy', 'Crime', 'Comedy']
The Hit Rate for this method is: 0.575
